# How to handle any images and link it with dataset

## Useful for the rescaling part

How to rescale an image (and eventually save the rescaled image)

In [5]:
%ls

Handling images and putting them into a dataset.ipynb*  IMGP0017.JPG*


In [10]:
from PIL import Image

divfactor = 1.2

# Rescaling a lambda image
im = Image.open("IMGP0017.JPG")
w, h = im.size
#print(type(im))
im = im.resize((int(w/divfactor), int(h/divfactor)))
im.save("resized" + ".JPG", "JPEG") # just to see the result
#print(type(im))

<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.Image.Image'>


## Useful for sliding part

Cropping an image:

In [11]:
im = Image.open("IMGP0017.JPG")
im = im.crop((0,0,1000,800))
im.save("cropped" + ".JPG", "JPEG") # just to see the result

## Useful to get a Dataset of Images

In [74]:
from PIL import Image

im = Image.open("IMGP0017.JPG")
# Convert into grayscale
im = im.convert("L")
# We have a list of Images
ims = [im.crop((0,0,1000,800)), im.crop((1000,800,2000,1600)), im.crop((2000,1600,3000,2400))]

TypeError: must be real number, not Image

The before-getting-a-Datase steps:

In [62]:
%matplotlib inline

In [63]:
import torch
import torchvision
import torchvision.transforms as transforms

# We define a transformation
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

### We want to get a Dataset from a list of Images!

Solution 1: converting into tensors, then using TensorDataset

In [61]:
from torch.utils.data import TensorDataset
import numpy as np

# 1/ Converting images into np arrays.. and then tensors
npims = []
for im in ims:
    npims.append(torch.from_numpy(np.array(im)))
    print(torch.from_numpy(np.array(im)).size())
    
# 2/ Converting into a Dataset
ds = TensorDataset(npims[0], npims[1], npims[2]) # for now, no transformation

torch.Size([800, 1000])
torch.Size([800, 1000])
torch.Size([800, 1000])


Solution 2: a homemade Dataset from images

In [86]:
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data.dataset import Dataset  # For custom datasets



class CustomDatasetFromImages(Dataset):
    """A dataset created from a list of PIL.Images. One minibatch of this 
    dataset is a tensor whose first dimension is its number of elements.
    A (complete) minibatch can be divided into 2 tensors:
    - a tensor of inputs, so a nbelements * 1 * 36 * 36 tensor (1 because grayscale,
    36*36 the size of the image).
    - a tensor of labels, so a nbelements tensor.
    """
    
    def convertalltotensor(self):
        """Convert the list of images into a pure tensor,
        with its elements all transformzs(normalized)"""
        #puretensor = torch.zeros(len(self.data),)
        l = []
        for im in self.data:
            tensored = self.to_tensor(im)
            l.append(tensored)
        l = torch.stack(l)
        self.data = l
    
    def __init__(self,ims,transform):
        self.data = ims
        self.transform = transform
        self.to_tensor = transforms.ToTensor() # We define the operation to transform an Image to a tensor
        # But ims is a list => minibatches will be list. We don't want that, 
        # we want minibatches to be tensors themselves.
        self.convertalltotensor()
        
    def __getitem__(self, index):
        """Important: defines one element (= the input as a tensor + the label (a simple nb))
        of a minibatch"""
        return (self.data[index], 0) # I want to return a tensor (the translated img) and a label
        
    def __len__(self):
        return len(self.data)
    
    
ds = CustomDatasetFromImages(ims, transform) 

Building a DataLoad from dataset

In [80]:
# 3/ 
dataloader = torch.utils.data.DataLoader(ds, batch_size=2,
                                          shuffle=True, num_workers=2)

What are really stored in the dataloader we've created ?

In [84]:
for data in dataloader:
        #print("data:",data)
        #print("data size:", len(data))
        #onlytorch = torch.Tensor(3, 36, 36)
        #torch.cat(data, out=onlytorch)()
        #print("onlytorch size", onlytorch.size())
        images, labels = data
        print("images:",images)
        print("images size:", images.size())
        print("labels:",labels)
        print("labels size:",labels.size())

images: tensor([[[[0.7843, 0.8078, 0.7804,  ..., 0.3294, 0.3255, 0.3647],
          [0.7059, 0.7294, 0.7412,  ..., 0.3216, 0.3333, 0.3725],
          [0.5255, 0.5804, 0.6314,  ..., 0.3373, 0.3490, 0.3686],
          ...,
          [0.1725, 0.1843, 0.1882,  ..., 0.6549, 0.6471, 0.6275],
          [0.1804, 0.1843, 0.1882,  ..., 0.6549, 0.6510, 0.6392],
          [0.1725, 0.1686, 0.1686,  ..., 0.6510, 0.6510, 0.6392]]],


        [[[0.5725, 0.5608, 0.5529,  ..., 0.9608, 0.9686, 0.9765],
          [0.5765, 0.5647, 0.5608,  ..., 0.9608, 0.9686, 0.9725],
          [0.5843, 0.5765, 0.5725,  ..., 0.9569, 0.9647, 0.9686],
          ...,
          [0.3451, 0.3529, 0.3490,  ..., 0.7608, 0.7569, 0.7529],
          [0.3373, 0.3412, 0.3451,  ..., 0.8078, 0.7843, 0.7569],
          [0.3490, 0.3569, 0.3569,  ..., 0.8235, 0.7961, 0.7647]]]])
images size: torch.Size([2, 1, 800, 1000])
labels: tensor([0, 0])
labels size: torch.Size([2])
images: tensor([[[[1.0000, 1.0000, 1.0000,  ..., 0.5255, 0.5216, 0.5

# The project

Getting chunks of the image with a sliding window

In [2]:
from PIL import Image

windims = (36,36) # dimensions on x and y axes of sliding window. Global variable


def get_imgchunks(image, strides=(1,1)): 
    """
    strides = (stridex, stridey)
    returns the Images but also the upper left positions of the 
    sliding windows. The stride is 1 (for now). But with stride 1, we
    have too many images!!! :O
    """
    chunks = []
    winpositions = [] # one position is a pair (x,y)
    w, h = image.size
    (curx, cury) = (0,0)
    sweptx, swepty = [False, False]
    # sliding on x-axis while not all is swept on x-axis:
    while not sweptx :  
        #print("curx:", curx)
        cury = 0
        swepty = False
        # sliding on y-axis while not all is swept on y-axis:
        while not swepty :
            #print("cury:", cury)
            chunks.append(image.crop((curx,cury,
                                   curx+windims[0],cury+windims[1])))
            winpositions.append((curx,cury))
            if cury+windims[1] == h:
                swepty = True
                if curx+windims[0] == w:
                    sweptx = True
            cury += strides[1]
            # But if the next windows overlaps with the end on y-axis,
            # we don't respect the stride we put but take the last window
            if cury+windims[1] > h:
                cury = h-windims[1]     
        curx += strides[0]
        # But if the next windows overlaps with the end on x-axis,
        # we don't respect the stride we put but take the last window
        #print(curx)
        if curx+windims[0] > w:
            curx = w-windims[0]
    return chunks, winpositions

Testing it, just to see the resulting cropped chunks:

In [3]:
im = Image.open("shrinked_IMGP0017.jpg")
(chunks, pos) = get_imgchunks(im, strides=(1,1))
print(pos)
#print(type(chunks[0]))
#for i in range(len(chunks)):
 #   chunks[i].save("crops_visualization/cropped" + str(i) + ".JPG", "JPEG") # just to see the result

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (10, 0), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (11, 0), (11, 1), (11, 2), (11, 3), (11, 4), (11, 5), (12, 0), (12, 1), (12, 2), (12, 3), (12, 4), (12, 5), (13, 0), (13, 1), (13, 2), (13, 3), (13, 4), (13, 5), (14, 0), (14, 1), (14, 2), (14, 3), (14, 4), (14, 5), (15, 0), (15, 1), (15, 2), (15, 3), (15, 4), (15, 5), (16, 0), (16, 1), (16, 2), (16, 3), (16, 4), (16, 5), (17, 0), (17, 1), (17, 2), (17, 3), (17, 4), (17, 5), (18, 0), (18, 1), (18, 2), (18, 3), (18, 4), (18, 5)]


Getting chunks of the image with a sliding window for different sizes

In [14]:
def get_imgchunks_atdiffscales(image, strides=(1,1), nbshrinkages=3, divfactor=1.2):
    """Returns the Image chunks at different scales. For now, stride=1
    We get 2 2d lists:
    - scaleschunks = first dimension is chunks for
    shrinked image with respectively a factor of divfactor, 2*divfactor, etc
    - scalespositions = first dimension is chunks positions for 
    shrinked image with respectively a factor of divfactor, 2*divfactor, etc
    """
    curdivfactor = divfactor
    curim = image
    w, h = image.size
    scaleschunks = []
    scalespositions = []
    for i in range(nbshrinkages+1):
        chunks, winpositions = get_imgchunks(curim, strides)
        scaleschunks.append(chunks)
        scalespositions.append(winpositions)
        curim = curim.resize((int(w/curdivfactor), int(h/curdivfactor)))
        curim.save("reduced"+str(i)+ ".JPG", "JPEG")
        curdivfactor *= 2
        curw, curh = curim.size
        # Of course we stop when the sliding window size is bigger than the image
        if curw<windims[0] or curh<windims[1]:
            break
    return scaleschunks, scalespositions

Testing get_imgchunks_atdiffscales:

In [15]:
im = Image.open("shrinked3_IMGP0017.jpg")
im = im.convert('L') # convert into grayscale
(scaleschunks, scalespositions) = get_imgchunks_atdiffscales(im, strides=(5,5), nbshrinkages=3, divfactor=2)
print(scalespositions)
for i in range(len(scaleschunks)):
    l = len(scaleschunks[i])
    print("number of positions:", l)
    #for j in range(l):
        #scaleschunks[i][j].save("crops_visualization/cropped" + str(i) + "-" + str(j) + ".JPG", "JPEG") # just to see the result

[[(0, 0), (0, 5), (0, 10), (0, 15), (0, 20), (0, 25), (0, 30), (0, 35), (0, 40), (0, 45), (0, 50), (0, 55), (0, 60), (0, 65), (0, 70), (0, 75), (0, 80), (0, 85), (0, 90), (0, 95), (0, 100), (0, 105), (0, 110), (0, 115), (0, 120), (0, 125), (0, 130), (0, 135), (0, 140), (0, 145), (0, 150), (0, 155), (0, 160), (0, 165), (0, 170), (0, 175), (0, 180), (0, 185), (0, 190), (0, 195), (0, 200), (0, 205), (0, 210), (0, 215), (0, 220), (0, 225), (0, 230), (0, 235), (0, 240), (0, 245), (0, 250), (0, 255), (0, 260), (0, 265), (0, 270), (0, 275), (0, 280), (0, 285), (0, 286), (5, 0), (5, 5), (5, 10), (5, 15), (5, 20), (5, 25), (5, 30), (5, 35), (5, 40), (5, 45), (5, 50), (5, 55), (5, 60), (5, 65), (5, 70), (5, 75), (5, 80), (5, 85), (5, 90), (5, 95), (5, 100), (5, 105), (5, 110), (5, 115), (5, 120), (5, 125), (5, 130), (5, 135), (5, 140), (5, 145), (5, 150), (5, 155), (5, 160), (5, 165), (5, 170), (5, 175), (5, 180), (5, 185), (5, 190), (5, 195), (5, 200), (5, 205), (5, 210), (5, 215), (5, 220), (5

In [35]:
tr = [1,2,3,4,5,6]
print(tr[-1])


6
